# Databricks: ETLs y Delta Lakes

## ¿Qué es un ETL?

Un ETL (Extract, Transform, Load) es un proceso fundamental en la ingeniería de datos. Consiste en extraer datos de diferentes fuentes, transformarlos para adecuarlos a las necesidades del negocio y cargarlos en un sistema de almacenamiento o análisis, como un Data Lake o un Data Warehouse.

### dbutils
`dbutils` es una colección de utilidades proporcionadas por Databricks para interactuar con el entorno, gestionar archivos, parámetros y flujos de trabajo. Es muy útil para la gestión de datos y automatización de tareas dentro de notebooks.

In [0]:
dbutils.help()

Dentro del catálogo de `dbutils` existen subsecciones como:
 - dbutils.fs: Para interactuar con el sistema de archivos
 - dbutils.secrets: Para usar secretos
 - dbutils.notebook: Para poder interactuar con otros notebooks
 - dbutils.widgets: Para poder parametrizar los notebooks


In [0]:
base_path = "dbfs:/Volumes/workspace/ceste/archivos/"
dbutils.fs.ls(base_path)

### Ingesta desde Distintos Orígenes

En los entornos de Big Data, los datos pueden venir en múltiples formatos: CSV, Parquet, JSON, entre otros. Cada formato tiene ventajas y desventajas en cuanto a compresión, velocidad de lectura/escritura y compatibilidad. Parquet, por ejemplo, es columnar y eficiente para grandes volúmenes.

**IMPORTANTE**: Antes de seguir subir a Databricks los ficheros del repositorio

#### Leer desde CSV

Leer datos desde un archivo CSV es una de las formas más comunes de ingesta en proyectos de datos. El formato CSV (Comma Separated Values) es ampliamente utilizado por su simplicidad y compatibilidad con la mayoría de las herramientas. Sin embargo, es importante tener en cuenta que no es el formato más eficiente para grandes volúmenes de datos, ya que no soporta tipos de datos complejos ni compresión nativa. Por eso, en entornos de Big Data, a menudo se prefiere convertir los datos a formatos como Parquet o Delta una vez cargados.

In [0]:
df_csv = spark.read.csv(
    path=base_path+"ventas.csv",
    header=True,
    inferSchema=True,
    sep=","
)

In [0]:
display(df_csv.head(5))

#### Leer desde parquet

El formato Parquet es un estándar de almacenamiento columnar ampliamente utilizado en Big Data. Su principal fortaleza es la eficiencia tanto en almacenamiento como en velocidad de lectura, especialmente cuando se trabaja con grandes volúmenes de datos y consultas sobre columnas específicas. Parquet permite compresión y soporta tipos de datos complejos, lo que lo hace ideal para análisis y procesamiento distribuido. Por ello, es habitual convertir datos de formatos como CSV a Parquet para optimizar el rendimiento y reducir costes en proyectos de análisis de datos.

In [0]:
df_parquet = spark.read.parquet(base_path+"ventas.parquet")

In [0]:
display(df_parquet)

#### Leer desde Json

El formato JSON (JavaScript Object Notation) es ampliamente utilizado para el intercambio de datos debido a su flexibilidad y legibilidad. Permite almacenar estructuras de datos complejas, como listas y diccionarios anidados. Sin embargo, en comparación con Parquet, JSON no es tan eficiente en almacenamiento ni en velocidad de procesamiento para grandes volúmenes de datos. Es útil cuando se requiere flexibilidad en la estructura de los datos o cuando los datos provienen de APIs y sistemas web.

In [0]:
df_json = spark.read.json(base_path+"ventas.json")

In [0]:
display(df_json)

#### Leer desde BD

La lectura de datos desde bases de datos (BD) es fundamental cuando los datos se encuentran almacenados en sistemas transaccionales o relacionales, como MySQL, SQL Server, PostgreSQL, entre otros. En entornos de Big Data, es común extraer datos de estas fuentes para integrarlos en un Data Lake o procesarlos con Spark. La conexión suele realizarse mediante JDBC, permitiendo ejecutar consultas SQL y cargar los resultados como DataFrames. Es importante considerar la seguridad, el rendimiento y la gestión de credenciales al conectar con bases de datos externas.

*La siguiente celda no va a hacer nada porque para funcionar necesitaríamos una de BD activa*

In [0]:
jdbc_url = "jdbc:mysql://<host>:<port>/<database>"
properties = {"user": "user", "password": "pwd"}

### Recordatorio: Leer con/sin esquema

Al leer datos en Spark, se puede dejar que el sistema infiera automáticamente el esquema (tipos de datos de cada columna) o definirlo explícitamente. Inferir el esquema es cómodo y rápido para exploraciones iniciales, pero puede ser más lento y propenso a errores si los datos son inconsistentes o si hay muchas columnas. Definir el esquema manualmente garantiza que los tipos de datos sean los esperados, mejora el rendimiento en la carga y ayuda a evitar problemas en etapas posteriores del procesamiento. Es una buena práctica definir el esquema en entornos productivos o cuando se requiere mayor control y robustez sobre los datos.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Definir el esquema manualmente
schema = StructType([
    # Nombre, Tipo de dato, Requerido
    StructField("id", StringType(), True),
    StructField("fecha", StringType(), True),
    StructField("producto", StringType(), True),
    StructField("cantidad", IntegerType(), True),
    StructField("precio", DoubleType(), True)
])

df_csv = spark.read.csv(
    path=base_path+"ventas.csv",
    header=True,
    schema=schema,
    sep=","
)

In [0]:
df_csv.printSchema()

## Delta Lake

### ¿Qué es Delta Lake?
Delta Lake es una capa de almacenamiento open source que se integra con Apache Spark y añade capacidades ACID (Atomicidad, Consistencia, Aislamiento, Durabilidad) a los Data Lakes. Permite versionado, time travel, y operaciones transaccionales sobre los datos.

### ¿Por qué es tan valioso el formato de Delta Table?
Guardar tablas en formato Delta permite aprovechar las ventajas de transacciones ACID, manejo de versiones, y optimización de consultas. Es ideal para entornos donde los datos cambian frecuentemente y se requiere trazabilidad.

Una Delta Table permite realizar operaciones ACID, mantener el histórico con time travel, gestionar versiones, optimizar el almacenamiento, y escalar en entornos de producción.

### Guardar una tabla

Guardar una tabla en Databricks puede hacerse de dos formas principales:

- Por path: Consiste en guardar los datos directamente en una ruta específica del sistema de archivos (por ejemplo, en DBFS) usando el formato Delta. Esta opción es flexible y permite gestionar los archivos de manera directa, moverlos entre entornos o integrarlos con otros sistemas que acceden por ruta.

- Por catálogo del metastore: Permite registrar la tabla en el catálogo de Databricks, lo que facilita su consulta mediante SQL y su integración con herramientas como Unity Catalog. Esta opción es ideal para entornos colaborativos, ya que permite a varios usuarios acceder a la misma tabla de forma controlada y segura, además de aprovechar funcionalidades avanzadas como el control de versiones, permisos y auditoría.

Ambas opciones aprovechan las ventajas del formato Delta, como las transacciones ACID, el versionado y la optimización de consultas. La elección entre una u otra depende de las necesidades de gestión, seguridad y acceso en el proyecto.

In [0]:
# Por path
df_csv.write.format("delta").mode("overwrite").save(base_path+"delta")

# Por catalogo
df_csv.write.format("delta").mode("overwrite").saveAsTable("ceste.productos")


### Leer una tabla

1. Lectura de una tabla Delta desde Spark: También puedes leer una tabla Delta directamente desde Spark usando el API de DataFrame:

In [0]:
df_delta = spark.read.format("delta").load(base_path+"delta")
df_delta.show()

Esto es útil cuando necesitas manipular los datos con Python, realizar transformaciones complejas, aplicar lógica de negocio o integrarlo en pipelines de procesamiento.

2. Consulta SQL sobre una tabla Delta: Puedes consultar una tabla Delta registrada en el catálogo usando SQL estándar. Por ejemplo:

In [0]:
%sql
SELECT * FROM ceste.productos;

Esto te permite aprovechar toda la potencia del lenguaje SQL para filtrar, agrupar, unir y analizar los datos almacenados en formato Delta. Es especialmente útil para usuarios que prefieren trabajar con SQL o para integraciones con herramientas de BI.

Diferencias y ventajas:
- Consultar por SQL es ideal para análisis exploratorio, dashboards y colaboración entre equipos.
- Leer con Spark DataFrame es más flexible para procesamiento avanzado, machine learning y automatización.
- Ambas formas aprovechan las ventajas de Delta Lake: transacciones ACID, versionado, rendimiento y escalabilidad.

### Modificar una tabla

Cuando trabajamos con tablas Delta, una de las grandes ventajas es la posibilidad de realizar operaciones transaccionales complejas de forma eficiente y segura como:
- Updates
- Deletes
- Merges

In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, base_path+"delta")

#### Update

Permite modificar el valor de una o varias columnas en las filas que cumplen una condición específica. Por ejemplo, se puede actualizar el nombre de un producto o corregir un valor erróneo en una tabla sin tener que reescribir todo el dataset. La sintaxis es similar a la de SQL, pero se realiza sobre la API de DeltaTable en Spark.

In [0]:
# UPDATE
delta_table.update(
    condition="id = 1",
    set={"producto": "'Ordenador'"}
)

#### Delete

Permite eliminar filas que cumplen una condición determinada. Es útil para depurar datos, eliminar registros obsoletos o cumplir con requisitos legales de borrado. Al igual que el update, el delete es transaccional y garantiza la integridad de la tabla.

In [0]:
# DELETE
delta_table.delete("precio > 150")

Ambas operaciones aprovechan las transacciones ACID de Delta Lake, lo que significa que los cambios son atómicos, consistentes, aislados y duraderos. Esto evita problemas de concurrencia y asegura que los datos siempre estén en un estado válido, incluso en entornos multiusuario o de procesamiento distribuido.

#### Merge

En este bloque de código se muestra cómo realizar un "merge" (también conocido como upsert) sobre una tabla Delta:

1. Se crea un DataFrame con nuevos datos o datos actualizados.
2. Luego, se utiliza el método `merge` de la API de Delta Lake para comparar los datos existentes en la tabla (target) con los nuevos datos (source) usando una condición de emparejamiento (en este caso, el campo id).
3. Si el `id` ya existe en la tabla, se actualizan todos los campos de ese registro (`whenMatchedUpdateAll`).
4. Si el `id` no existe, se inserta el nuevo registro (`whenNotMatchedInsertAll`).

In [0]:
# Nuevos datos a insertar/actualizar
columns = ["id", "fecha", "producto", "cantidad", "precio"]

nuevos_datos = [(3, "2025-05-24", "Monitor", 1, 179.99), (4, "2025-05-24", "Impresora", 2, 89.99)]
df_updates = spark.createDataFrame(nuevos_datos, columns)


delta_table.alias("target").merge(
    df_updates.alias("source"),
    "target.id = source.id") \
  .whenMatchedUpdateAll() \
  .whenNotMatchedInsertAll() \
  .execute()

Este tipo de operación es fundamental en escenarios de integración incremental de datos, donde periódicamente llegan nuevos registros o actualizaciones y queremos mantener la tabla Delta siempre actualizada y sin duplicados.

Ventajas de usar merge en Delta Lake:

- Permite mantener la integridad y consistencia de los datos.
- Facilita la implementación de pipelines de datos incrementales.
- Aprovecha las transacciones ACID de Delta Lake, evitando problemas de concurrencia o corrupción de datos.
- Es mucho más eficiente y sencillo que realizar operaciones manuales de actualización e inserción por separado.

### Time Travel

El Time Travel en Delta Lake es una funcionalidad que permite consultar versiones anteriores de una tabla Delta. Cada vez que se realiza una operación de escritura (insert, update, delete, merge), Delta Lake crea una nueva versión de la tabla, manteniendo el historial de cambios.

**¿Para qué sirve el Time Travel?**
- Recuperar datos borrados o modificados accidentalmente.
- Auditar cambios y analizar cómo han evolucionado los datos a lo largo del tiempo.
- Comparar el estado de la tabla en diferentes momentos.
- Reproducir experimentos o análisis sobre datos históricos.

**¿Cómo se usa?**
Puedes acceder a una versión anterior de la tabla especificando el número de versión (`versionAsOf`) o una marca de tiempo (`timestampAsOf`) al leer los datos:

**Ventajas:**
- No necesitas mantener copias manuales de los datos para auditoría o recuperación.
- Todas las operaciones de Time Travel son transaccionales y consistentes.
- Facilita la trazabilidad y el cumplimiento normativo en entornos empresariales.

In [0]:
display(delta_table.toDF())

In [0]:
# Ver historial
display(delta_table.history())

In [0]:
# Leer versión anterior
df_old = spark.read.format("delta").option("versionAsOf", 0).load(base_path+"delta")
display(df_old)

In [0]:
# Leer la tabla tal como estaba en una fecha concreta
df_moment = spark.read.format("delta").option("timestampAsOf", "2025-05-27 10:00:00").load(base_path+"delta")

### Optimización de Tablas

La optimización de tablas en Delta Lake es clave para mejorar el rendimiento de las consultas y reducir el coste de almacenamiento en entornos de Big Data. Existen dos técnicas principales:

- **Compactación (OPTIMIZE):** Consiste en reducir el número de archivos pequeños que se generan tras múltiples escrituras o actualizaciones. Al compactar, se agrupan estos archivos en otros más grandes, lo que acelera las lecturas y reduce la sobrecarga de gestión de archivos en el sistema distribuido.

- **Z-Ordering:** Es una técnica de ordenación física de los datos en disco basada en una o varias columnas clave. Al aplicar Z-Ordering, los datos se almacenan de forma que las filas con valores similares en las columnas seleccionadas queden físicamente próximas. Esto mejora notablemente el rendimiento de las consultas filtradas por esas columnas, ya que minimiza la cantidad de datos que Spark necesita leer.

**Ventajas de la optimización:**
- Consultas más rápidas y eficientes, especialmente en grandes volúmenes de datos.
- Menor latencia en dashboards y análisis interactivos.
- Reducción de costes de almacenamiento y procesamiento.
- Mejor aprovechamiento de los recursos del cluster.

**Cuándo optimizar:**
- Tras cargas masivas de datos o procesos ETL frecuentes.
- Cuando se detecta degradación en el rendimiento de las consultas.
- Antes de ejecutar análisis críticos o dashboards de negocio.

En resumen, la optimización periódica de las tablas Delta es una buena práctica para mantener el entorno ágil, eficiente y escalable.

In [0]:
# Optimizar tabla
spark.sql("OPTIMIZE ceste.productos")
# Ordenar físicamente por "id"
spark.sql("OPTIMIZE ceste.productos ZORDER BY id")

## Ejercicios

1. Crea un DF con datos de clientes. Para ello puedes utilizar la información que te doy y la funcion `list(zip(X,Y))`

In [0]:
nombres = ["Juan", "María", "Carlos", "Ana", "Luis", "Carmen", "José", "Laura", "Pedro", "Lucía", "Miguel", "Elena", "Javier", "Sofía", "Antonio", "Marta", "Manuel", "Isabel", "Francisco", "Patricia"]
edades = [25, 30, 22, 28, 35, 27, 40, 32, 24, 29, 33, 26, 31, 23, 36, 21, 34, 38, 37, 20]



2. Añade la columna indice. Para ello ayudate de la funcion `monotonically_increasing_id`

In [0]:
from pyspark.sql.functions import monotonically_increasing_id


3. Guardar como Delta Table (por path). Utiliza la variable `base_path`

4. Actualiza la edad de un cliente

In [0]:
from delta.tables import DeltaTable



5. Insertar un nuevo cliente

In [0]:
from pyspark.sql.functions import current_timestamp



6. Consulta el historial de la tabla. Posteriormente lee una version anterior con Time Travel

In [0]:
# Ver historial de cambios


# Leer la versión inicial (version 0)


7. Registrar como tabla del metastore

8. Borra todos los registros

%md
9. Usa widgets para seleccionar dinámicamente la tabla Delta que creamos al principio del notebook ("delta") y mostrar su contenido

%md
10. Time travel con widgets, utiliza un widget tipo texto para poder leer dinamicamente la primera version de la tabla delta inicial ("delta").

%md
11. Atomicidad de Delta Table: Haz una demostracion de la atomicidad en delta Tables, prepara una insercion de dos filas donde la primera vaya a ser correcta y la segunda erronea. Seguidamente comprueba que no se inserto nada

In [0]:
from pyspark.sql.types import *

# Este batch contiene un error: la segunda fila tiene un precio como string
data_erronea = [
    ("10", "2025-05-28", "Tablet", 2, 299.99),
    ("11", "2025-05-28", "Teclado", 3, "caro")  # Error intencionado
]

%md
12. exceptAll() - Permite comparar 2 versiones de dfs y ver sus diferencias.

carga dos versiones de tu tabla delta en dos dfs distintos y aplicalo

In [0]:

df_actual=
df_anterior=



df_actual.exceptAll(df_anterior).display()